In [2]:
from IPython.display import display, display_html, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import threading
from datetime import datetime

In [3]:
population = pd.read_csv("population.csv")
military = pd.read_csv("military.csv")
electricity = pd.read_csv("electricity.csv")
gdp = pd.read_csv("gdp.csv")
prisoners = pd.read_csv("prisoners.csv")
roads = pd.read_csv("roads.csv")
rain = pd.read_csv("rain.csv")
households = pd.read_csv("households.csv")
births = pd.read_csv("births.csv")

combined = pd.merge(population, military, on='Country')
combined = pd.merge(combined, electricity, on='Country')
combined = pd.merge(combined, gdp, on='Country')
combined = pd.merge(combined, prisoners, on='Country')
combined = pd.merge(combined, roads, on='Country')
combined = pd.merge(combined, rain, on='Country')
combined = pd.merge(combined, households, on='Country')
combined = pd.merge(combined, births, on='Country')
combined['Alphabetical'] = range(1, len(combined) + 1)

numeric_entries = ['Population', 'Active Military', 'Electricity Cons.', 'GDP',
                   'Prison Pop', 'Road Network', 'Avg. Precipitation', 'Households', 'Births']

for columns in numeric_entries:
    # Remove commas from the entire column before conversion
    combined[columns] = combined[columns].str.replace(',', '')
    # Now convert the column to numeric
    combined[columns] = pd.to_numeric(combined[columns], errors='coerce')


combined

Country  Population  Active Military  Electricity Cons.      GDP  \
0       Albania     2761785             8000               7400    23032   
1     Argentina    46654581            72100             127000   621833   
2       Armenia     2981200            44800               6100    24540   
3     Australia    26461912            58600             237000  1687713   
4       Austria     9154514            22050              69000   526182   
..          ...         ...              ...                ...      ...   
106     Uruguay     3444263            21000              14000    76244   
107  Uzbekistan    36599764            48000              58000    90392   
108     Vietnam   100300000           482000             229000   433356   
109      Zambia    19610769            15100              14000    29536   
110    Zimbabwe    15178979            29000               8300    32424   

     Prison Pop  Road Network  Avg. Precipitation  Households   Births  \
0          4807          3945                1485      862512    29289   
1        114074        281290                 591    13925275   629391   
2          2128          7700                 562      837896    33700   
3         41155        873573                 534    10220186   298496   
4          8798        126400                1110     3933516    84647   
..          ...           ...                 ...         ...      ...   
106       14347         77732                1300     1269537    35892   
107       29000        209496                 206     6586983   802875   
108      125697        570448                1821    25503951  1462623   
109       24000         67671                1020     3485901   672000   
110       22114         97267                 657     3795226   488807   

     Alphabetical  
0               1  
1               2  
2               3  
3               4  
4               5  
..            ...  
106           107  
107           108  
108           109  
109           110  
110           111  

[111 rows x 11 columns]

In [4]:
matrix0 = combined[['Population', 'Active Military', 'Electricity Cons.', 'GDP',
                   'Prison Pop', 'Road Network', 'Avg. Precipitation', 'Households', 'Births']].corr()
matrix0

Population  Active Military  Electricity Cons.       GDP  \
Population            1.000000         0.841549           0.775161  0.564486   
Active Military       0.841549         1.000000           0.837901  0.717358   
Electricity Cons.     0.775161         0.837901           1.000000  0.861619   
GDP                   0.564486         0.717358           0.861619  1.000000   
Prison Pop            0.706122         0.838865           0.888814  0.902179   
Road Network          0.837095         0.838243           0.793891  0.835339   
Avg. Precipitation   -0.044600        -0.098933          -0.107642 -0.106570   
Households            0.960728         0.870589           0.907682  0.677507   
Births                0.905026         0.687525           0.489277  0.362680   

                    Prison Pop  Road Network  Avg. Precipitation  Households  \
Population            0.706122      0.837095           -0.044600    0.960728   
Active Military       0.838865      0.838243           -0.098933    0.870589   
Electricity Cons.     0.888814      0.793891           -0.107642    0.907682   
GDP                   0.902179      0.835339           -0.106570    0.677507   
Prison Pop            1.000000      0.870632           -0.063002    0.795707   
Road Network          0.870632      1.000000           -0.098846    0.826389   
Avg. Precipitation   -0.063002     -0.098846            1.000000   -0.065764   
Households            0.795707      0.826389           -0.065764    1.000000   
Births                0.510462      0.734039           -0.031392    0.765707   

                      Births  
Population          0.905026  
Active Military     0.687525  
Electricity Cons.   0.489277  
GDP                 0.362680  
Prison Pop          0.510462  
Road Network        0.734039  
Avg. Precipitation -0.031392  
Households          0.765707  
Births              1.000000

In [5]:
matrix1 = combined[['Active Military', 'Electricity Cons.', 'GDP', 'Prison Pop', 'Road Network']].corr()
np.fill_diagonal(matrix1.values, np.nan)
matrix1

Active Military  Electricity Cons.       GDP  Prison Pop  \
Active Military                NaN           0.837901  0.717358    0.838865   
Electricity Cons.         0.837901                NaN  0.861619    0.888814   
GDP                       0.717358           0.861619       NaN    0.902179   
Prison Pop                0.838865           0.888814  0.902179         NaN   
Road Network              0.838243           0.793891  0.835339    0.870632   

                   Road Network  
Active Military        0.838243  
Electricity Cons.      0.793891  
GDP                    0.835339  
Prison Pop             0.870632  
Road Network                NaN

In [6]:
matrix2 = combined[['Active Military', 'Electricity Cons.', 
             'GDP', 'Prison Pop', 'Road Network', 'Avg. Precipitation', 'Alphabetical']].corr()
np.fill_diagonal(matrix2.values, np.nan)
matrix2

Active Military  Electricity Cons.       GDP  Prison Pop  \
Active Military                 NaN           0.837901  0.717358    0.838865   
Electricity Cons.          0.837901                NaN  0.861619    0.888814   
GDP                        0.717358           0.861619       NaN    0.902179   
Prison Pop                 0.838865           0.888814  0.902179         NaN   
Road Network               0.838243           0.793891  0.835339    0.870632   
Avg. Precipitation        -0.098933          -0.107642 -0.106570   -0.063002   
Alphabetical               0.046219          -0.031817  0.040935    0.035152   

                    Road Network  Avg. Precipitation  Alphabetical  
Active Military         0.838243           -0.098933      0.046219  
Electricity Cons.       0.793891           -0.107642     -0.031817  
GDP                     0.835339           -0.106570      0.040935  
Prison Pop              0.870632           -0.063002      0.035152  
Road Network                 NaN           -0.098846      0.005959  
Avg. Precipitation     -0.098846                 NaN     -0.048467  
Alphabetical            0.005959           -0.048467           NaN

In [8]:
matrix3 = combined[['Electricity Cons.', 
             'GDP', 'Prison Pop', 'Road Network', 'Avg. Precipitation', 'Alphabetical', 'Households', 'Births']].corr()
np.fill_diagonal(matrix3.values, np.nan)
matrix3

Electricity Cons.       GDP  Prison Pop  Road Network  \
Electricity Cons.                 NaN  0.861619    0.888814      0.793891   
GDP                          0.861619       NaN    0.902179      0.835339   
Prison Pop                   0.888814  0.902179         NaN      0.870632   
Road Network                 0.793891  0.835339    0.870632           NaN   
Avg. Precipitation          -0.107642 -0.106570   -0.063002     -0.098846   
Alphabetical                -0.031817  0.040935    0.035152      0.005959   
Households                   0.907682  0.677507    0.795707      0.826389   
Births                       0.489277  0.362680    0.510462      0.734039   

                    Avg. Precipitation  Alphabetical  Households    Births  
Electricity Cons.            -0.107642     -0.031817    0.907682  0.489277  
GDP                          -0.106570      0.040935    0.677507  0.362680  
Prison Pop                   -0.063002      0.035152    0.795707  0.510462  
Road Network                 -0.098846      0.005959    0.826389  0.734039  
Avg. Precipitation                 NaN     -0.048467   -0.065764 -0.031392  
Alphabetical                 -0.048467           NaN   -0.073550 -0.026846  
Households                   -0.065764     -0.073550         NaN  0.765707  
Births                       -0.031392     -0.026846    0.765707       NaN

In [12]:
corr_list = combined[['Electricity Cons.', 
             'GDP', 'Prison Pop', 'Road Network', 'Avg. Precipitation', 'Alphabetical', 'Households', 
                      'Births']].corrwith(combined['Population'])

# Compute the outer product of the series with itself
table = np.outer(pd.Series(corr_list), pd.Series(corr_list))

# Convert the result into a pandas DataFrame
table = pd.DataFrame(table, 
                                       index=corr_list.index, 
                                       columns=corr_list.index)
np.fill_diagonal(table.values, np.nan)
table

Electricity Cons.       GDP  Prison Pop  Road Network  \
Electricity Cons.                 NaN  0.437567    0.547358      0.648883   
GDP                          0.437567       NaN    0.398596      0.472528   
Prison Pop                   0.547358  0.398596         NaN      0.591091   
Road Network                 0.648883  0.472528    0.591091           NaN   
Avg. Precipitation          -0.034572 -0.025176   -0.031493     -0.037334   
Alphabetical                -0.052585 -0.038293   -0.047901     -0.056786   
Households                   0.744719  0.542317    0.678391      0.804221   
Births                       0.701541  0.510875    0.639059      0.757593   

                    Avg. Precipitation  Alphabetical  Households    Births  
Electricity Cons.            -0.034572     -0.052585    0.744719  0.701541  
GDP                          -0.025176     -0.038293    0.542317  0.510875  
Prison Pop                   -0.031493     -0.047901    0.678391  0.639059  
Road Network                 -0.037334     -0.056786    0.804221  0.757593  
Avg. Precipitation                 NaN      0.003026   -0.042848 -0.040364  
Alphabetical                  0.003026           NaN   -0.065173 -0.061395  
Households                   -0.042848     -0.065173         NaN  0.869484  
Births                       -0.040364     -0.061395    0.869484       NaN

In [16]:
series1 = table.melt()['value']
series2 = matrix3.melt()['value']

# Compute Pearson correlation coefficient and p-value
series1.corr(series2)


0.909673904790244

In [17]:
series_diff = series2 - series1
series_diff.mean()

0.06559804260247866

In [19]:
series_diff.std()

0.17612953570457984

In [7]:
matrix1_means = matrix1.mean(axis=1)
matrix1_means


Active Military      0.808092
Electricity Cons.    0.845556
GDP                  0.829124
Prison Pop           0.875122
Road Network         0.834526
dtype: float64

In [8]:
matrix2_means = matrix2.mean(axis=1)
matrix2_means

Active Military       0.529942
Electricity Cons.     0.540461
GDP                   0.541810
Prison Pop            0.578773
Road Network          0.540870
Avg. Precipitation   -0.087243
Alphabetical          0.007997
dtype: float64

In [14]:
matrix3_means = matrix3.mean(axis=1)
matrix3_means

Population           0.791436
Electricity Cons.    0.786074
GDP                  0.700635
Prison Pop           0.778986
Road Network         0.816231
Households           0.822287
Births               0.627865
dtype: float64

In [15]:
# equally weighted index
combined_n = combined[['Active Military', 'Electricity Cons.', 'GDP', 'Prison Pop', 'Road Network', 'Avg. Precipitation', 
                       'Alphabetical', 'Households', 'Births']].div(combined[[
                       'Active Military', 'Electricity Cons.', 'GDP', 'Prison Pop', 'Road Network', 'Avg. Precipitation', 
                       'Alphabetical', 'Households', 'Births']].max())

normalized_index1 = pd.DataFrame((combined_n[combined_n.columns[0]] + combined_n[combined_n.columns[1]] + 
                             combined_n[combined_n.columns[2]] + combined_n[combined_n.columns[3]] +
                             combined_n[combined_n.columns[4]])/5)
normalized_index2 = pd.DataFrame((combined_n[combined_n.columns[0]] + combined_n[combined_n.columns[1]] + 
                             combined_n[combined_n.columns[2]] + combined_n[combined_n.columns[3]] +
                             combined_n[combined_n.columns[4]] + combined_n[combined_n.columns[5]] +
                             combined_n[combined_n.columns[6]])/7)
normalized_index3 = pd.DataFrame((combined_n[combined_n.columns[0]] + combined_n[combined_n.columns[1]] + 
                             combined_n[combined_n.columns[2]] + combined_n[combined_n.columns[3]] +
                             combined_n[combined_n.columns[4]] + combined_n[combined_n.columns[7]] +
                             combined_n[combined_n.columns[8]])/9)

normalized1 = normalized_index1[normalized_index1.columns[0]].corr(combined['Population'])
normalized2 = normalized_index2[normalized_index2.columns[0]].corr(combined['Population'])
normalized3 = normalized_index3[normalized_index3.columns[0]].corr(combined['Population'])


print("Correlation of normalized weight index 1: ", normalized1)
print("Correlation of normalized weight index 2: ", normalized2)
print("Correlation of normalized weight index 3: ", normalized3)
combined_n

Correlation of normalized weight index 1:  0.806354023480265
Correlation of normalized weight index 2:  0.6674665062898734
Correlation of normalized weight index 3:  0.890099213873108


Active Military  Electricity Cons.       GDP  Prison Pop  Road Network  \
0           0.003661           0.000948  0.000855    0.002720      0.000577   
1           0.032998           0.016270  0.023074    0.064551      0.041172   
2           0.020503           0.000781  0.000911    0.001204      0.001127   
3           0.026819           0.030361  0.062625    0.023288      0.127865   
4           0.010092           0.008839  0.019525    0.004978      0.018501   
..               ...                ...       ...         ...           ...   
106         0.009611           0.001793  0.002829    0.008118      0.011378   
107         0.021968           0.007430  0.003354    0.016410      0.030664   
108         0.220595           0.029336  0.016080    0.071128      0.083496   
109         0.006911           0.001793  0.001096    0.013581      0.009905   
110         0.013272           0.001063  0.001203    0.012514      0.014237   

     Avg. Precipitation  Alphabetical  Households    Births  
0              0.458333      0.009009    0.001650  0.001267  
1              0.182407      0.018018    0.026642  0.027230  
2              0.173457      0.027027    0.001603  0.001458  
3              0.164815      0.036036    0.019553  0.012914  
4              0.342593      0.045045    0.007526  0.003662  
..                  ...           ...         ...       ...  
106            0.401235      0.963964    0.002429  0.001553  
107            0.063580      0.972973    0.012602  0.034735  
108            0.562037      0.981982    0.048794  0.063279  
109            0.314815      0.990991    0.006669  0.029073  
110            0.202778      1.000000    0.007261  0.021148  

[111 rows x 9 columns]

In [17]:
# CWA index
cwa_index1 = pd.DataFrame(combined_n[combined_n.columns[0]] *  matrix1_means.iloc[0] +
                          combined_n[combined_n.columns[1]] *  matrix1_means.iloc[1] +
                          combined_n[combined_n.columns[2]] *  matrix1_means.iloc[2] +
                          combined_n[combined_n.columns[3]] *  matrix1_means.iloc[3] +
                          combined_n[combined_n.columns[4]] *  matrix1_means.iloc[4])

cwa_index2 = pd.DataFrame(combined_n[combined_n.columns[0]] *  matrix2_means.iloc[0] +
                          combined_n[combined_n.columns[1]] *  matrix2_means.iloc[1] +
                          combined_n[combined_n.columns[2]] *  matrix2_means.iloc[2] +
                          combined_n[combined_n.columns[3]] *  matrix2_means.iloc[3] +
                          combined_n[combined_n.columns[4]] *  matrix2_means.iloc[4] +
                          combined_n[combined_n.columns[5]] *  matrix2_means.iloc[5] +
                          combined_n[combined_n.columns[6]] *  matrix2_means.iloc[6])

cwa_index3 = pd.DataFrame(combined_n[combined_n.columns[0]] *  matrix3_means.iloc[0] +
                          combined_n[combined_n.columns[1]] *  matrix3_means.iloc[1] +
                          combined_n[combined_n.columns[2]] *  matrix3_means.iloc[2] +
                          combined_n[combined_n.columns[3]] *  matrix3_means.iloc[3] +
                          combined_n[combined_n.columns[4]] *  matrix3_means.iloc[4] +
                          #combined_n[combined_n.columns[5]] *  matrix3_means.iloc[5] +
                          #combined_n[combined_n.columns[6]] *  matrix3_means.iloc[6] +
                          combined_n[combined_n.columns[7]] *  matrix3_means.iloc[5] +
                          combined_n[combined_n.columns[8]] *  matrix3_means.iloc[6])

cwa1 = cwa_index1[cwa_index1.col[0]].corr(combined['Population'])
cwa3 = cwa_index3[cwa_index3.cumns[0]].corr(combined['Population'])
cwa2 = cwa_index2[cwa_index2.columnsolumns[0]].corr(combined['Population'])

print("Correlation of normalized weight index 1: ", cwa1)
print("Correlation of normalized weight index 2: ", cwa2)
print("Correlation of normalized weight index 3: ", cwa3)

Correlation of normalized weight index 1:  0.8050508670293001
Correlation of normalized weight index 2:  0.800492355417846
Correlation of normalized weight index 3:  0.8866660081775984
